# Bi-LSTM Model Retrain

## Data

In [1]:
import os

import pandas as pd

from os.path import join
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

In [66]:
import datetime
# Get the current date and time
now = datetime.datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M")

In [71]:
# ROOT = '/home/sdc/DR_DemandForecast/DemandForecast/'
ROOT = '/home/meta/Desktop/DR_DemandForecast/DemandForecast'
DES_DIR = f'../model/{timestamp}'

if not os.path.exists(DES_DIR):
    os.makedirs(DES_DIR)

### Load from DB

In [41]:
def db_conn():
    # Load the environment variables from the .env file
    env_file = join(ROOT, 'src', '.env')
    load_dotenv(env_file)

    # Get the values of host, user, pswd, db, and schema from the environment variables
    DBHOST = os.getenv('host')
    DBUSER = os.getenv('user')
    DBPSWD = os.getenv('pswd')
    DBNAME = os.getenv('db')
    SCHEMA = 'public'

    # Use the values as needed
    engine = create_engine(f"postgresql://{DBUSER}:{DBPSWD}@{DBHOST}/{DBNAME}?options=-csearch_path%3D{SCHEMA}", echo=False)
    conn = engine.connect()
    
    return conn

In [42]:
def get_dpr(
    conn: object,
    limit: int=None
):
    select_query = """SELECT "Date", "Period", "Demand", "TCL", "TransmissionLoss", "Solar"
    FROM public."RealTimeDPR"
    ORDER BY "Date" DESC, "Period" DESC
"""
    if limit:
        select_query += f"    LIMIT {limit}\n;"
    else:
        select_query += ";"
        
    # print(select_query)
    
    dpr = pd.read_sql(select_query, conn)
    
    dpr.fillna(0, inplace=True)
    
    dpr["TotalDemand"] = dpr["Demand"] + dpr["TCL"] + dpr["TransmissionLoss"] + dpr["Solar"]
    
    return dpr

### Process Data

In [5]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib
import os
import glob

In [58]:
def process_dpr(
    total_demand: pd.Series,
    newest_dir: str
):
    
    '''
    ## Process Total Demand
    
    `newest_dir` is needed for loading the latest scaler file.
    '''
    
    total_demand = total_demand.copy()

    ''' Scale '''
    scaler_files = glob.glob(os.path.join(newest_dir, "*.pkl"))
    scaler = joblib.load(scaler_files[0])
    total_demand = scaler.fit_transform(total_demand.values.reshape(-1,1))

    ''' Split '''

    weeks = 0.5
    days = 0
    periods = 0
    
    if len(total_demand) < 1:
        raise ValueError("Dataset is empty.")
    
    # 1 <len_x> + 1 Y is 1 unit of data.
    # There should be at least 6 units of data. This is call a group.
    #   - 3 for training, 1 for validation, 2 for test.
    
    len_x = int(periods + (days * 48) + (weeks * 7 * 48))
    # print(len(total_demand), len_x)
    
    train_units = 2
    valid_units = 1
    test_units = 1
    
    if len(total_demand) < (len_x + 1) * (test_units + valid_units + train_units):
        raise ValueError(f"Not enough data for training. {len(total_demand)} < {(len_x + 1) * 6}")
    
    max_groups = len(total_demand) // ((len_x + 1) * (test_units + valid_units + train_units))
    
    idx_test = - max_groups * test_units * (len_x + 1)
    idx_vali = - max_groups * (test_units + valid_units) * (len_x + 1)
    # print(idx_test, idx_vali)
    
    test  = total_demand[idx_test:]
    valid = total_demand[idx_vali:idx_test]
    train = total_demand[:idx_vali]
                        
    print(f"train: {len(train)}, validate: {len(valid)}, test: {len(test)}")
    
    
    ''' Reshape '''
    
    train_x, train_Y = [], []
    for i in range(len(train)-len_x):
        train_x.append(train[i:(i+len_x)])
        train_Y.extend(train[i + len_x])
        
    valid_x, valid_Y = [], []
    for i in range(len(valid)-len_x):
        valid_x.append(valid[i:(i+len_x)])
        valid_Y.extend(valid[i + len_x])
        
    test_x, test_Y = [], []
    for i in range(len(test)-len_x):
        test_x.append(test[i:(i+len_x)])
        test_Y.extend(test[i + len_x])
        
    return np.array(train_x), np.array(train_Y), np.array(valid_x), np.array(valid_Y), np.array(test_x), np.array(test_Y), scaler
        


## Load and Retrain Model

In [7]:
import tensorflow as tf
import keras as tk

import glob

2024-04-30 13:52:43.051497: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-30 13:52:43.051555: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-30 13:52:43.053331: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-30 13:52:43.061668: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-30 13:52:44.059591: W tensorflow/compiler/tf2

In [8]:
def load_model(
    newest: str
):
    
    model_files = glob.glob(os.path.join(newest, "*.keras"))

    # Sort the list of model files by modification time (most recent first)
    model_files.sort(key=os.path.getmtime, reverse=True)

    # Select the most recent model file
    most_recent_model_file = model_files[0]

    # Load the selected model
    model = tk.models.load_model(most_recent_model_file)
    
    return model

In [45]:
# if __name__ == "__main__":
    
# Load new data
conn = db_conn()
dpr = get_dpr(
    conn, 
    # limit=336
)
    
    

In [59]:
# Find newest directory
newest_dir = max(glob.glob(os.path.join(ROOT, 'model', '*/')), key=os.path.getmtime)

train_x, train_Y, validate_x, validate_Y, test_x, test_Y, scaler = process_dpr(dpr["TotalDemand"], newest_dir)

train: 424, validate: 169, test: 169


In [47]:
# Load the most recent model
model = load_model(newest_dir)

In [64]:
from sklearn.metrics import mean_squared_error

# Generate benchmark predictions and Accuracy
test_predictions = model.predict(test_x)

i_test_predict = scaler.inverse_transform(test_predictions)
i_test_Y = scaler.inverse_transform([test_Y])

test_score_p = np.sqrt(mean_squared_error(i_test_Y[0], i_test_predict[:,0]))

print(f"Before tuning RMSE: {test_score_p}")

1/1 [==============================] - 0s 17ms/step
Before tuning RMSE: 95.51470312499987


In [65]:
# Retrain model
model.fit(train_x, train_Y, epochs=100, batch_size=32, verbose=0, validation_data=(validate_x, validate_Y))

# Generate predictions and Accuracy
test_predictions = model.predict(test_x)

i_test_predict = scaler.inverse_transform(test_predictions)
i_test_Y = scaler.inverse_transform([test_Y])

test_score_a = np.sqrt(mean_squared_error(i_test_Y[0], i_test_predict[:,0]))

print(f"After tuning RMSE: {test_score_a}")

2024-04-30 14:42:27.792320: I external/local_xla/xla/service/service.cc:168] XLA service 0x7b4058bdf670 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-30 14:42:27.792343: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2024-04-30 14:42:27.796494: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1714459347.876120 2337070 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 [==============================] - 0s 17ms/step
After tuning RMSE: 13.528374999999869


In [72]:
# Save model if accuracy is better
if test_score_a < test_score_p:
    # Define the filename
    filename = f'{DES_DIR}/bi-lstm.keras'

    # Save the model
    model.save(filename)
    # Save a report of the model
    report = {
        'timestamp': timestamp,
        'data': {
            'lookback': 168,
            'weeks': 0.5,
            'days': 0,
            'periods': 0,
        }
    }

    import json

    with open(f'{DES_DIR}/report.json', 'w') as f:
        json.dump(report, f, indent=2)
    

In [ ]:
conn.close()